In [7]:
# import the opencv library
import cv2
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [8]:
# import our model
from keras.models import load_model

In [9]:
# Other dependencies
import numpy as np
import tensorflow as tf
import os
from PIL import Image

In [10]:
# Crop center of the image feed
def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))

# Crop largest square from the image feed
def crop_max_square(pil_img):
    return crop_center(pil_img, min(pil_img.size), min(pil_img.size))

# Crop image around face
def crop_face(pil_img, x, y, w, h):
    return pil_img.crop((x, y, x+w, y+h))

In [11]:
# define a video capture object
stream = cv2.VideoCapture(0)
model = load_model('emotion_model.h5')

while(True):
      
    # Capture the video frame by frame
    res, frame = stream.read()
    height, width, channel = frame.shape
    
    #Convert the captured frame into RGB
    im = Image.fromarray(frame, 'RGB')
    
    # Find face in the frame
    face = face_haar_cascade.detectMultiScale(frame)
    x = face[0][0]
    y = face[0][1]
    w = face[0][2]
    h = face[0][3]
    
    #Rectangle around the face
    cv2.rectangle(img = frame, pt1 = (x,y), pt2 = (x+w, y+h), color = (255, 0, 0), thickness= 2)
    
    # Cropping image around face
    im_new = crop_face(im, x, y, w, h)
    
    # Resizing into dimensions you used while training
    im_new_new = crop_max_square(im_new)
    im_new_new = im_new_new.resize((48,48))
    img_array = np.array(im_new_new)
    
    #Expand dimensions to match the 4D Tensor shape.
    img_array = np.expand_dims(img_array, axis=1)
    img_array = img_array.reshape(-1, 48, 48, 1)
    
    # Scale to match train data
    img_array = img_array/255
    
    #Calling the predict function using keras
    prediction = model.predict(img_array)
    print(prediction)
    
    # Taking only the first of three arrays
    array = prediction[0]
    
    # Find location of the highest probability
    percent = round(max(array),2)
    predict = np.where(array == max(array))
    emotion = predict[0]
    
    # Part of the code that predicts the emotion from the prediction array
    if(emotion == 0):
        output = "Angry"
    elif(emotion == 1):
        output = "Disgust"
    elif(emotion == 2):
        output = "Fear"
    elif(emotion == 3):
        output = "Happy"
    elif(emotion == 4):
        output = "Sad"
    elif(emotion == 5):
        output = "Surprise"
    elif(emotion == 6):
        output = "Neutral"
        
    print(output)
    
    cv2.putText(frame, f'{output} {percent}%', (0,50), cv2.FONT_HERSHEY_DUPLEX, 1, (255,255,255), 2)
    # Display the resulting frame
    cv2.imshow('Predict Emotion', frame)
      
    # the 'q' button is set as the quitting button
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
# After the loop release the cap object
stream.release()

# Destroy all the windows
cv2.destroyAllWindows()

1/1 [==============================] - 0s 107ms/step
[[5.89041822e-02 5.50922996e-04 6.21329411e-04 7.03639686e-01
  4.36232500e-02 4.51054220e-04 1.92209512e-01]
 [4.00219299e-03 6.32923111e-05 5.04332002e-06 7.68412292e-01
  4.39334176e-02 1.58836984e-03 1.81995437e-01]
 [1.86508764e-02 7.90881677e-05 2.74755639e-05 6.54661179e-01
  1.21001236e-01 3.53141455e-04 2.05227062e-01]]
Happy
1/1 [==============================] - 0s 23ms/step
[[1.2020113e-01 2.7605696e-04 1.4718273e-03 7.0836341e-01 3.8249489e-02
  3.0586883e-04 1.3113223e-01]
 [1.0284838e-02 7.8021832e-05 1.1276864e-05 8.0563420e-01 5.1594637e-02
  1.2082052e-03 1.3118878e-01]
 [3.0706653e-02 1.2325490e-04 5.2748430e-05 6.4707983e-01 1.6235541e-01
  4.4417253e-04 1.5923803e-01]]
Happy
1/1 [==============================] - 0s 23ms/step
[[7.9003893e-02 3.6063982e-04 7.5796089e-04 7.2758251e-01 4.2184498e-02
  3.0465343e-04 1.4980584e-01]
 [9.9942368e-03 8.6721448e-05 1.4893846e-05 7.5171649e-01 6.8837345e-02
  1.0430500e-03

IndexError: tuple index out of range

: 

In [ ]:
# cap=cv2.VideoCapture(1)while cap.isOpened():
#     res,frame=cap.read()
#     height, width , channel = frame.shape
#     #---------------------------------------------------------------------------
#     # Creating an Overlay window to write prediction and cofidence
#     sub_img = frame[0:int(height/6),0:int(width)]
#     black_rect = np.ones(sub_img.shape, dtype=np.uint8)*0
#     res = cv2.addWeighted(sub_img, 0.77, black_rect,0.23, 0)
#     FONT = cv2.FONT_HERSHEY_SIMPLEX
#     FONT_SCALE = 0.8
#     FONT_THICKNESS = 2
#     lable_color = (10, 10, 255)
#     lable = "Emotion Detection made by Abhishek"
#     lable_dimension = cv2.getTextSize(lable,FONT ,FONT_SCALE,FONT_THICKNESS)[0]
#     textX = int((res.shape[1] - lable_dimension[0]) / 2)
#     textY = int((res.shape[0] + lable_dimension[1]) / 2)
#     cv2.putText(res, lable, (textX,textY), FONT, FONT_SCALE, (0,0,0), FONT_THICKNESS)# prediction part --------------------------------------------------------------------------gray_image= cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     faces = face_haar_cascade.detectMultiScale(gray_image )
#     try:
#         for (x,y, w, h) in faces:
#             cv2.rectangle(frame, pt1 = (x,y),pt2 = (x+w, y+h), color = (255,0,0),thickness =  2)
#             roi_gray = gray_image[y-5:y+h+5,x-5:x+w+5]
#             roi_gray=cv2.resize(roi_gray,(48,48))
#             image_pixels = img_to_array(roi_gray)
#             image_pixels = np.expand_dims(image_pixels, axis = 0)
#             image_pixels /= 255
#             predictions = model.predict(image_pixels)
#             max_index = np.argmax(predictions[0])
#             emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
#             emotion_prediction = emotion_detection[max_index]
#             cv2.putText(res, "Sentiment: {}".format(emotion_prediction), (0,textY+22+5), FONT,0.7, lable_color,2)
#             lable_violation = 'Confidence: {}'.format(str(np.round(np.max(predictions[0])*100,1))+ "%")
#             violation_text_dimension = cv2.getTextSize(lable_violation,FONT,FONT_SCALE,FONT_THICKNESS )[0]
#             violation_x_axis = int(res.shape[1]- violation_text_dimension[0])
#             cv2.putText(res, lable_violation, (violation_x_axis,textY+22+5), FONT,0.7, lable_color,2)
#     except :
#         pass
#     frame[0:int(height/6),0:int(width)] = res
#     cv2.imshow('frame', frame)if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
# cv2.destroyAllWindows
